In [9]:
import openpyxl as xl
from decimal import Decimal, getcontext

class Result:
    getcontext().prec = 9
    target_cols = [4, 11, 12, 22]

    def __init__(self, n_of_digits):
        '''self.n_of_digits - кол-во знаков порсле запятой'''
        self.n_of_digits = n_of_digits 
        self.result = []
        
    def load_prepare_make(self, link):
        '''Загружает Excel-таблицу, обрабатывает её, полноценно создает атрибут result'''
        wb = xl.load_workbook(f'таблицы/{link}.xlsx', data_only = True)
        sheet = wb.active
        self.__total_gos_price = Decimal(sheet.cell(row = 6, column = 7).value)
        for i in range(24, sheet.max_row):
            fin_row = []
            for j in Result.target_cols:
                if j == 4:
                    fin_row.append(str(Result.first_col_creator(sheet.cell(row=i,column=j).value)))
                else:
                    fin_row.append(str(sheet.cell(row = i, column = j).value))
            self.result.append(fin_row)
        for i in self.result:
            i.append(Decimal(i[1]) * Decimal(i[2]))
        for i in self.result:
            i.append(Decimal(i[3]) / Decimal(i[2]) - 1)   

    #################################################################

    @staticmethod
    def first_col_creator(txt):
        '''Метод формирует первый элемент каждой строки списка .result'''
        txt = txt.lower().split()
        ml_num = ['мл', '№']
        first_col = []
        for i in txt:
            if i != 'мг/мл':
                for j in ml_num:
                    if j in i:
                        first_col.append(i.replace(j,''))
        first_col = [float(i) for i in first_col]
        res = 1
        for i in first_col:
            res *= i
        return res

    #################################################################

    def defining_consts(self):
        '''Определение общей суммы и суммы строки'''
        self.our_total = sum([i[4] for i in self.result])
        self.rows_sum = [i[4] for i in self.result]
        self.difference = self.__total_gos_price - self.our_total

    def update(self):
        '''Обновляет значения суммы строки и процента до предела реестра'''
        for i in range(len(self.result)): #сумма строки
            self.result[i][4] = Decimal(self.result[i][1]) * Decimal(self.result[i][2])
            self.result[i][5] = (Decimal(self.result[i][3]) / Decimal(self.result[i][2])) - 1
        self.defining_consts()

    #################################################################

    def count_up_to_n_decimal_places(self,index):
        '''Подбивает строку под n-знаков'''
        price_of_ml = Decimal(self.result[index][2]) / Decimal(self.result[index][0])
        price_of_ml = str(price_of_ml).split('.')[0] + '.' + str(price_of_ml).split('.')[1][:self.n_of_digits]
        new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        while new_full_price != Decimal(str(new_full_price).split('.')[0] + '.' + str(new_full_price).split('.')[1][:2]) and Decimal(price_of_ml) > 0:
            price_of_ml = str(Decimal(price_of_ml) - Decimal(str(1 / ( 10 ** self.n_of_digits))))
            new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        self.result[index][2] = str(new_full_price)
        self.update()

    def minimize_diff(self):
        '''Минимизирует разницу self.difference'''
        all_allowed_vars = []                          # создание списка цен за упаковку, которые подогнаны под n знаков
        delta = Decimal('0.01')
        for i in range(len(self.result)):              # пробежимся по всей таблице и просчитаем каждую строку
            price = Decimal(self.result[i][2])
            const = Decimal(self.result[i][0])
            max_price = Decimal(self.result[i][3])
            lst = []
            while price < max_price:
                price += delta
                price_of_ml = price / const
                if len(str(price_of_ml).split('.')[1]) <= self.n_of_digits:
                    lst.append(price)
            all_allowed_vars.append(lst)
        
        diffs_list = []
        for counter, list in enumerate(all_allowed_vars):
            if len(list) != 0:
                amount = Decimal(self.result[counter][1])
                row_sum = Decimal(self.result[counter][4])
                for price in list:
                    rn_diff = self.difference
                    rn_diff -= (price * amount) - row_sum
                    if rn_diff >= 0:
                        min_diff_list = [counter, price, rn_diff]
                    else:
                        break
            if 'min_diff_list' in locals():
                diffs_list.append(min_diff_list)
        diffs_list.sort(key = lambda x: x[2])
        best_index = diffs_list[0][0]
        best_price = diffs_list[0][1]
        self.result[best_index][2] = str(best_price)
        self.update() 

    def divide_to_2_rows(self, index):
        pass # coming soon
    
    #################################################################
    
    def price_price_output(self):
        for j, i in enumerate(self.result, start = 1):
            price, const = Decimal(i[2]), Decimal(i[0])
            print(f'{j}) {price}---{price / const}')

    def final_print(self):
        '''Метод финальной печати .result'''
        self.update()
        for i in self.result:
            print(str(i))
        print('Цена госзакупки: ' + str(self.__total_gos_price))
        print('Цена контракта:  ' + str(self.our_total))
        print('Разница:         ' + str(self.difference))
        print()


In [ ]:
# Result(n) n - кол-во знаков после запятой 
# load_prepare_make('name') name - имя файла без .xlsx
# count_up_to_n_decimal_places(index) index - номер строки, которую надо просчитать 
# update() пересчитывает все суммы, это делается автоматически, но для безопасности надо пересчитывать еще раз
# res.minimize_diff() - минимизирует разницу и соответственно подбивает цену под n знаков 
# final_print() вывод таблицы и всех сумм
# price_price_output() выводит в формате    цена за упаковку --- цена за миллилитр

In [12]:
res = Result(2) # в скобках количество знаков после запятой в цене за миллилитр
res.load_prepare_make('0038')  # в кавыках имя файла без .xlsx
res.update() 
for i in range(len(res.result)):
    res.count_up_to_n_decimal_places(i)  # просчет каждой строки по индексу под n знаков после запятой в цене за мл
res.update()
res.minimize_diff()
# res.minimize_diff()
res.final_print()  # функция финального вывода таблицы
res.price_price_output()  # вывод в формате:   цена за упаковку --- цена за миллилитр

['6400.0', '744', '448.000', '626.35', Decimal('333312.000'), Decimal('0.39810268')]
['6400.0', '197', '576.000', '626.35', Decimal('113472.000'), Decimal('0.08741319')]
['6000.0', '547', '540.000', '590.35', Decimal('295380.000'), Decimal('0.09324074')]
['6000.0', '92', '540.000', '590.35', Decimal('49680.000'), Decimal('0.09324074')]
Цена госзакупки: 809832
Цена контракта:  791844.000
Разница:         17988.000

1) 448.000---0.07
2) 576.000---0.09
3) 540.000---0.09
4) 540.000---0.09
